In [1]:
from lib.utils import *
from lib.models import *
from lib.ekyn import *
from lib.env import *
from lib.datasets import *

In [2]:
trainloader = DataLoader(Dataset2p0(dir='../data/w9_ss/train',labels='../data/w9_ss/y_train.pt'),batch_size=32,shuffle=True)
devloader = DataLoader(Dataset2p0(dir='../data/w9_ss/dev',labels='../data/w9_ss/y_dev.pt'),batch_size=32,shuffle=True)

In [ ]:
subjects = [load_eeg_label_pair(id=id,condition=condition,zero_pad=True,windowsize=9) for id in get_ekyn_ids() for condition in ['Vehicle','PF']]

In [ ]:
8639 // 8640

In [ ]:
8644 % 8640  

In [ ]:
276480 / (8640*2)

In [ ]:
class Subjectset(Dataset):
    def __init__(self,subjects):
        self.Xs = [subject[0] for subject in subjects]
        self.ys = [subject[1] for subject in subjects]
    def __len__(self):
        return 276480

    def __getitem__(self, idx):
        return (self.Xs[idx // 8640][(idx % 8640) :(idx % 8640) + 9].flatten(),self.ys[idx // 8640][idx % 8640])

In [ ]:
subjectloader = DataLoader(Subjectset(subjects=subjects),batch_size=32,shuffle=True)

In [ ]:
X,y = next(iter(subjectloader))

In [ ]:
class Windowset(Dataset):
    def __init__(self,X,y):
        self.X = cat([zeros(4,5000),X,zeros(4,5000)])
        self.y = y

    def __len__(self):
        return len(self.y)

    def __getitem__(self, idx):
        return (self.X[idx:idx+9].flatten(),self.y[idx])

In [ ]:
trainloader = DataLoader(Windowset(X_train,y_train),batch_size=32,shuffle=True)

In [ ]:

trainloader = DataLoader(TensorDataset(X_train,y_train),batch_size=32,shuffle=True)
devloader = DataLoader(TensorDataset(X_dev,y_dev),batch_size=32,shuffle=True)
testloader = DataLoader(TensorDataset(X_test,y_test),batch_size=32,shuffle=True)

In [3]:
class A(nn.Module):
    def __init__(self) -> None:
        super().__init__()
        self.fc1 = nn.Linear(45000,10)
        self.fc2 = nn.Linear(10,3)

    def forward(self,x):
        x = self.fc1(x)
        x = relu(x)
        x = self.fc2(x)
        return x
model = A().to(DEVICE)

In [4]:
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(),lr=3e-4)
params = sum([p.flatten().size()[0] for p in list(model.parameters())])
print("Params: ",params)

Params:  450043


In [5]:
import time
start_time = time.time()
model.train()
loss_tr_total = 0
for (X_tr,y_tr) in tqdm(trainloader):
    X_tr,y_tr = X_tr.to(DEVICE),y_tr.to(DEVICE)
    logits = model(X_tr)
    loss = criterion(logits,y_tr)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    loss_tr_total += loss.item()
print("--- %s seconds ---" % (time.time() - start_time))

  7%|▋         | 379/5184 [00:10<02:13, 36.08it/s]


KeyboardInterrupt: 